# Acceleration Example

### Imports

In [1]:
import org.apache.spark.streaming.{Seconds, Minutes, StreamingContext}
import org.apache.kafka.common.serialization.{BytesDeserializer, StringDeserializer}
import org.apache.spark.streaming.kafka010.KafkaUtils
import org.apache.spark.streaming.kafka010.LocationStrategies.PreferConsistent
import org.apache.spark.streaming.kafka010.ConsumerStrategies.Subscribe
import com.fasterxml.jackson.databind.ObjectMapper
import com.fasterxml.jackson.module.scala.DefaultScalaModule
import com.fasterxml.jackson.module.scala.experimental.ScalaObjectMapper
import org.apache.kafka.clients.producer.{KafkaProducer, ProducerConfig, ProducerRecord}
import org.apache.spark.sql.types.StructType
import collection.JavaConverters.mapAsJavaMapConverter

Compile Error: <console>:26: error: object kafka is not a member of package org.apache
       import org.apache.kafka.common.serialization.{BytesDeserializer, StringDeserializer}
                         ^
<console>:33: error: object kafka is not a member of package org.apache
       import org.apache.kafka.clients.producer.{KafkaProducer, ProducerConfig, ProducerRecord}
                         ^


### Create Streaming Context

In [2]:
val ssc = new StreamingContext(sc, Seconds(1))
ssc.remember(Minutes(1))

Compile Error: <console>:27: error: not found: type StreamingContext
       val ssc = new StreamingContext(sc, Seconds(1))
                     ^
<console>:27: error: not found: value Seconds
       val ssc = new StreamingContext(sc, Seconds(1))
                                          ^


### Setup Kafka input stream

In [3]:
val consumerParams = Map[String, Object](
  "bootstrap.servers" -> "kafka:9092",
  "key.deserializer" -> classOf[BytesDeserializer],
  "value.deserializer" -> classOf[StringDeserializer],
  "group.id" -> "spark-notebook",
  "auto.offset.reset" -> "earliest",
  "enable.auto.commit" -> (false: java.lang.Boolean)
)

val topics = Array("sample_topic")
val stream = KafkaUtils.createDirectStream[String, String](
  ssc,
  PreferConsistent,
  Subscribe[String, String](topics, consumerParams)
)

Compile Error: <console>:27: error: not found: type BytesDeserializer
         "key.deserializer" -> classOf[BytesDeserializer],
                                       ^
<console>:28: error: not found: type StringDeserializer
         "value.deserializer" -> classOf[StringDeserializer],
                                         ^
<console>:35: error: not found: value KafkaUtils
       val stream = KafkaUtils.createDirectStream[String, String](
                    ^
<console>:36: error: not found: value ssc
         ssc,
         ^
<console>:37: error: not found: value PreferConsistent
         PreferConsistent,
         ^
<console>:38: error: not found: value Subscribe
         Subscribe[String, String](topics, consumerParams)
         ^


### Expected Input Schema

In [8]:
val schema = new StructType().add("time", "long").add("x", "float").add("y", "float").add("z", "float")

Compile Error: <console>:25: error: not found: type StructType
       val schema = new StructType().add("time", "long").add("x", "float").add("y", "float").add("z", "float")
                        ^


### Stream Processing

In [9]:
stream.foreachRDD { rdd =>
  spark.read.schema(schema).json(rdd.map(_.value())).createOrReplaceTempView("locations")
  spark.sql("select avg(x) as x, avg(y) as y, avg(z) as z, min(timestamp) as timestamp from locations").toJSON.foreachPartition {
    partition =>

      val producerParams = Map[String, Object](
        ProducerConfig.BOOTSTRAP_SERVERS_CONFIG -> "kafka:9092",
        ProducerConfig.VALUE_SERIALIZER_CLASS_CONFIG -> "org.apache.kafka.common.serialization.StringSerializer",
        ProducerConfig.KEY_SERIALIZER_CLASS_CONFIG -> "org.apache.kafka.common.serialization.StringSerializer"
      )
 
      val producer = new KafkaProducer[String, String](producerParams.asJava)
      
      partition.foreach { s =>
        if (s != "{}")
            println(s)
            producer.send(new ProducerRecord[String, String]("sample_topic", s))
            //acceleration
      }
      
      producer.close()
  }
}

Compile Error: <console>:28: error: not found: value stream
       stream.foreachRDD { rdd =>
       ^
<console>:29: error: package schema is not a value
         spark.read.schema(schema).json(rdd.map(_.value())).createOrReplaceTempView("locations")
                           ^
<console>:34: error: not found: value ProducerConfig
               ProducerConfig.BOOTSTRAP_SERVERS_CONFIG -> "kafka:9092",
               ^
<console>:35: error: not found: value ProducerConfig
               ProducerConfig.VALUE_SERIALIZER_CLASS_CONFIG -> "org.apache.kafka.common.serialization.StringSerializer",
               ^
<console>:36: error: not found: value ProducerConfig
               ProducerConfig.KEY_SERIALIZER_CLASS_CONFIG -> "org.apache.kafka.common.serialization.StringSerializer"
               ^
<console>:39: error: not found: type KafkaProducer
             val producer = new KafkaProducer[String, String](producerParams.asJava)
                                ^
<console>:39: error: value as

### Start stream

In [6]:
ssc.start()

Compile Error: <console>:26: error: not found: value ssc
       ssc.start()
       ^


### Lets see what we really read

In [10]:
%%SQL
select * from locations

Magic SQL failed to execute with error: 
Table or view not found: locations; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [locations], [], false


### Stop stream

In [7]:
StreamingContext.getActive.foreach { _.stop(stopSparkContext = false) }

Exception in thread "streaming-job-executor-0" java.lang.Error: java.lang.InterruptedException
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1148)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:617)
	at java.lang.Thread.run(Thread.java:745)
Caused by: java.lang.InterruptedException
	at java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:998)
	at java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1304)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:202)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:218)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:153)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:623)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:1873)
	at org.apache.spark.SparkContext.runJob

#### Verify the contents in Kafka using the console consumer

The following command line tools can help print the contents to the console.
```sh
./bin/kafka-console-consumer.sh --topic acceleration --bootstrap-server localhost:9092
```